In [1]:
import dask.dataframe as dd
import pandas as pd
import sys
import os

In [2]:
df = dd.read_csv('/home/peterkad/pkadbioinfproject/testguppy/data/METTL3/WT_1_HQcalls_RMS/run.transcriptome.aligned.eventalign.tsv_processed_perpos_mean_fixedcontig.tsv',sep='\t', blocksize=1e9)

In [3]:
def process(contig,position,data, label):
	print("Processing {}_{}".format(contig,position))
	chr = str(contig)
	subs = data[data['contig'] == chr]
	data['position'] += 3 # Add 3 nt to each position 
	data['Pos'] = data['contig'].astype(str) + '_' + data['position'].astype(str) # Unique column
	data['sample'] = label #Add label
	windows = pd.DataFrame() # Create an empty DataFrame
	#Create windows file
	
	subs = data[data['contig'] == chr]
	mod = int(position)
	window = [mod + i for i in range(-7, 8)]
	for wind in window:
		subs2 = subs[subs['position'] == wind]
		if len(subs2) > 1:
			subs2['modification'] = chr + '_' + str(mod)
			subs2['reference'] = str(wind - mod)
			windows = pd.concat([windows, subs2])
	return windows

In [4]:
df = df.map_partitions(lambda df: df[df['contig'] == 'ENST00000007516.8'] )



In [5]:
df = df.map_partitions(process("ENST00000007516.8","490",df,"WT_1"))

Processing ENST00000007516.8_490


3012192

In [17]:
# Filter the data
df = df[df['contig'] == 'ENST00000007516.8']

df.compute().head()

,contig,position,reference_kmer,read_name,event_level_mean
48240510,ENST00000007516.8,9,CCTGG,4d9aa0ae-fb1a-48bc-b663-7519806ca801,86.755
48240511,ENST00000007516.8,10,CTGGG,1018b194-8f36-4814-99bb-7191026dfeff,108.320
48240512,ENST00000007516.8,10,CTGGG,4d9aa0ae-fb1a-48bc-b663-7519806ca801,104.340
48240513,ENST00000007516.8,11,TGGGT,1018b194-8f36-4814-99bb-7191026dfeff,115.820
48240514,ENST00000007516.8,11,TGGGT,1e8a11f3-0905-4f57-88a9-78b114a8d423,107.935
